In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from pycaret.classification import *
from src.utils import *
from src.calcEMA import *
from src.myenv import *
import plotly.express as px


In [ ]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
# lightgbm  xgboost
estimator = 'xgboost'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 2.0
label = 'status'
regression_times = 24 * 30 * 2  # horas
regression_profit_and_loss = 24
# numeric_features=['open', 'high', 'low', 'volume', 'close', 'rsi']
numeric_features = ['open', 'high', 'low', 'volume', 'close', ]
_calc_rsi = True

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [ ]:
use_cols = date_features + numeric_features
print(use_cols)
all_data = read_data(f'{datadir}/{symbol}', all_cols=None, use_cols=use_cols)

# Filter all data
all_data = all_data[(all_data['open_time'] >= start_train_date)].copy()

print(all_data.info())

In [ ]:
if _calc_rsi:
  all_data = calc_RSI(all_data)
  numeric_features.append('rsi')
  all_data.dropna(inplace=True)
print(all_data.info())

In [ ]:
all_cols = date_features + numeric_features
print('All Columns: ', all_cols)
all_data[all_cols]
#all_data = all_data[[all_cols]].copy()

In [ ]:
def regress_until_diff(data: pd.DataFrame, diff_percent: float, max_regression_profit_and_loss=6):
    data['close_shift_x'] = 0.0
    data['diff_shift_x'] = 0.0
    data['shift_x'] = 0
    data[label] = 'ESTAVEL'
    for row_nu in range(1, data.shape[0]):
        diff = 0
        i = 1

        while (abs(diff) <= diff_percent):
            if (i > max_regression_profit_and_loss) or ((row_nu + i) >= data.shape[0]):
                break

            close = data.iloc[row_nu:row_nu + 1]['close'].values[0]
            close_px = data.iloc[row_nu + i:row_nu + i + 1]['close'].values[0]
            diff = -100 * (close - close_px) / close
            # print(f'ROW_NU: {row_nu} - regresssion_times: {i} - diff: {diff}')
            i += 1

        data['close_shift_x'].iloc[row_nu:row_nu + 1] = close_px
        data['diff_shift_x'].iloc[row_nu:row_nu + 1] = diff
        data['shift_x'].iloc[row_nu:row_nu + 1] = i - 1 if i == max_regression_profit_and_loss + 1 else i

        if diff >= diff_percent:
            data[label].iloc[row_nu:row_nu + 1] = 'SOBE_' + str(diff_percent)
            
        elif diff <= -diff_percent:
            data[label].iloc[row_nu:row_nu + 1] = 'CAI_' + str(diff_percent)

    return data.drop(columns=['close_shift_x', 'diff_shift_x', 'shift_x'])

In [ ]:
all_data = regress_until_diff(all_data, stop_loss, regression_profit_and_loss)

all_data

In [ ]:
for nf in numeric_features.copy():
    for i in range(1, regression_times + 1):
        col = nf + "_" + str(i)
        all_data[col] = all_data[nf].shift(i)
        numeric_features.append(col)

all_data.dropna(inplace=True)
all_data.info()

In [ ]:
train_data = all_data[(all_data['open_time'] >= start_train_date) & (all_data['open_time'] < start_test_date)]
train_data = train_data.sort_values(date_features)
train_data

In [ ]:
setup = setup(train_data,
              train_size=0.7,
              target=label,
              numeric_features=numeric_features,
              date_features=['open_time'],
              create_date_columns=["hour", "day", "month"],
              fold_strategy='timeseries',
              fold=3,
              session_id=123,
              normalize=True,
              use_gpu=False,
              verbose=True,
              n_jobs=20,
              )

In [ ]:
#Accuracy	AUC	Recall	Prec.	F1	Kappa	MCC
#model_name = 'lightgbm' 'xgboost'
if _compare_models:
  best = setup.compare_models()
  estimator = best.__class__.__name__
else:
  best = setup.create_model(estimator)

best

In [ ]:
test_data = all_data[all_data['open_time'] >= start_test_date]
test_data = test_data.sort_values(date_features)

In [ ]:
# predict on test set
holdout_pred = predict_model(best)
print(holdout_pred['prediction_score'].mean())

holdout_pred[[label, 'prediction_score']].groupby(label).mean()

In [ ]:
predict = predict_model(best, data=test_data.drop(columns=[label]))
predict[label] = test_data[label]
predict['_score'] = predict['prediction_label'] == predict[label]
print('Score Mean:', predict['_score'].mean())

In [ ]:
predict[[label, '_score']].groupby(label).mean()

In [ ]:
final_predict = finalize_model(best) # data=test_data.sort_values(date_features).drop(columns=[label]))

In [ ]:
_predict = predict_model(final_predict, data=test_data.sort_values(date_features).drop(columns=[label]))
_predict[label] = test_data[label]
_predict['_score'] = _predict['prediction_label'] == _predict[label]
print('Score Mean:', _predict['_score'].mean())

In [ ]:
_predict[[label, '_score']].groupby(label).mean()

In [ ]:
for i in range(1, 9999):
  filename = f'{symbol}_{estimator}_SL_{stop_loss}_RT_{regression_times}_RPL_{regression_profit_and_loss}_{i}'
  print('Model file name: ', filename + '.pkl')
  if os.path.exists(filename + '.pkl'):
    continue
  else:    
    save_model(final_predict, filename)
    break